In [ ]:
#EKPHRASIS by Bob Tianqi Wei, Shayne Shen, UC Berkeley, 2024

from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

# Load the pre-trained VGG16 model without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional layers of VGG16 to prevent them from being updated during training
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = base_model.output
x = Flatten()(x)  # Flatten the feature maps
x = Dense(128, activation='relu')(x)  # Fully connected layer
predictions = Dense(1, activation='sigmoid')(x)  # Output layer with sigmoid activation for binary classification

# Define the new model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Display the model architecture
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,926,209 (68.38 MB)

 Trainable params: 3,211,521 (12.25 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to the dataset, containing 'class_0' and 'class_1' folders
data_dir = r"/Users/bobwei/Downloads/composition_dataset"  # Change to your own dataset path

# Use ImageDataGenerator to automatically split the dataset into training and validation sets
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # 80% for training, 20% for validation

# Load training data
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),  # VGG16 requires input images to be 224x224
    batch_size=32,
    class_mode='binary',  # Binary classification task
    subset='training'  # Use 80% of the data for training
)

# Load validation data
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'  # Use 20% of the data for validation
)

Found 132 images belonging to 2 classes.
Found 32 images belonging to 2 classes.


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,  # Number of steps per epoch
    validation_data=validation_generator,  # Validation data
    validation_steps=validation_generator.samples // validation_generator.batch_size,  # Number of validation steps
    epochs=10  # Set the number of epochs according to your dataset
)

Epoch 1/10


/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 15s 4s/step - accuracy: 0.5865 - loss: 1.7769 - val_accuracy: 0.3125 - val_loss: 1.2607
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 1.6257  
Epoch 3/10


2024-10-20 15:33:22.168705: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/opt/anaconda3/lib/python3.12/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)
2024-10-20 15:33:22.176360: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


4/4 ━━━━━━━━━━━━━━━━━━━━ 12s 4s/step - accuracy: 0.7035 - loss: 0.6567 - val_accuracy: 0.6875 - val_loss: 0.7321
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7188 - loss: 0.6104
Epoch 5/10


2024-10-20 15:33:36.667073: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


4/4 ━━━━━━━━━━━━━━━━━━━━ 12s 3s/step - accuracy: 0.8162 - loss: 0.4696 - val_accuracy: 0.6250 - val_loss: 0.6264
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8750 - loss: 0.4266
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 12s 3s/step - accuracy: 0.8398 - loss: 0.4105 - val_accuracy: 0.6875 - val_loss: 0.6621
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7812 - loss: 0.4646
Epoch 9/10


2024-10-20 15:34:06.879422: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


4/4 ━━━━━━━━━━━━━━━━━━━━ 13s 3s/step - accuracy: 0.8684 - loss: 0.3740 - val_accuracy: 0.5625 - val_loss: 0.7208
Epoch 10/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9062 - loss: 0.2921


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Define a function to predict the class and confidence of a new image
def predict_custom_image(image_path):
    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(224, 224))  # Resize image to 224x224
    img_array = image.img_to_array(img)  # Convert image to array
    img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions to match model input
    img_array /= 255.0  # Normalize pixel values to the range [0, 1]
    
    # Make a prediction
    prediction = model.predict(img_array)
    
    # Output the prediction result
    if prediction >= 0.5:
        print(f"The image is classified as class_1 with confidence {prediction[0][0]:.4f}")
    else:
        print(f"The image is classified as class_0 with confidence {1 - prediction[0][0]:.4f}")

# Use the trained model to predict a new image
image_path = '/Users/bobwei/Downloads/image.png'  # Replace with the path of the image to predict
predict_custom_image(image_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
The image is classified as class_1 with confidence 0.9701


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
import os

# Define a function to predict the class and confidence of a single image
def predict_custom_image(image_path):
    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(224, 224))  # Resize image to 224x224
    img_array = image.img_to_array(img)  # Convert image to array
    img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions to match model input
    img_array /= 255.0  # Normalize pixel values to the range [0, 1]
    
    # Make a prediction
    prediction = model.predict(img_array)
    
    # Return the prediction result
    if prediction >= 0.5:
        return f"class_1 with confidence {prediction[0][0]:.4f}"
    else:
        return f"class_0 with confidence {1 - prediction[0][0]:.4f}"

# Batch prediction for multiple images
image_dir = '/Users/bobwei/Downloads/'  # Replace with the folder path containing the images
for i in range(1, 13):  # Loop through images 1 to 12
    image_path = os.path.join(image_dir, f'image{i}.png')  # Generate the path for each image
    result = predict_custom_image(image_path)
    print(f"Image {i}: {result}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
Image 1: class_1 with confidence 0.7597
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
Image 2: class_1 with confidence 0.9135
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step
Image 3: class_1 with confidence 0.5711
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
Image 4: class_1 with confidence 0.5896
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
Image 5: class_1 with confidence 0.8091
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
Image 6: class_0 with confidence 0.5330
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
Image 7: class_0 with confidence 0.5701
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
Image 8: class_1 with confidence 0.8882
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
Image 9: class_1 with confidence 0.8158
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
Image 10: class_1 with confidence 0.6653
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
Image 11: class_0 with confidence 0.6316
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
Image 12: class_1 with confidence 0.5903
